In [1]:
import numpy as np

## Downsampling

In [2]:
def max_downsampling(matrix, kernel=3):
    stride = kernel
    h, w = matrix.shape
    output_dim = (h // stride, w // stride)
    output = np.zeros(output_dim, dtype=int)

    for i in range(0, h-kernel+1, stride):
        for j in range(0, w-kernel+1, stride):
            region = matrix[i:i+kernel, j:j+kernel]
            output[i//stride, j//stride] = np.max(region)
    
    return output

def median_downsampling(matrix, kernel=3):
    stride = kernel
    h, w = matrix.shape
    output_dim = (h // stride, w // stride)
    output = np.zeros(output_dim, dtype=int)

    for i in range(0, h-kernel+1, stride):
        for j in range(0, w-kernel+1, stride):
            region = matrix[i:i+kernel, j:j+kernel]
            output[i//stride, j//stride] = np.median(region)
    
    return output

def mean_downsampling(matrix, kernel=3):
    stride = kernel
    h, w = matrix.shape
    output_dim = (h // stride, w // stride)
    output = np.zeros(output_dim, dtype=int)

    for i in range(0, h-kernel+1, stride):
        for j in range(0, w-kernel+1, stride):
            region = matrix[i:i+kernel, j:j+kernel]
            output[i//stride, j//stride] = np.mean(region)
    
    return output

In [3]:
x = matrix = np.random.randint(0, 10, size=(10, 10))
print(x)

[[4 3 2 5 5 1 7 3 1 3]
 [4 3 4 0 7 9 2 6 7 7]
 [2 4 9 4 2 6 7 7 8 6]
 [1 6 6 2 0 7 5 7 6 4]
 [5 9 2 4 1 0 1 0 2 1]
 [5 0 3 5 7 3 0 7 0 9]
 [1 6 8 0 4 2 3 0 0 9]
 [8 4 9 5 5 2 6 3 6 1]
 [9 9 6 5 1 2 6 6 7 8]
 [1 5 1 9 3 9 2 7 7 2]]


In [4]:
max_dw = max_downsampling(x)
print(f"Max Downsampling")
print(max_dw,"\n")

median_dw = median_downsampling(x)
print(f"Median Downsampling")
print(median_dw,"\n")

mean_dw = mean_downsampling(x)
print(f"Mean Downsampling")
print(mean_dw,"\n")

Max Downsampling
[[9 9 8]
 [9 7 7]
 [9 5 7]] 

Median Downsampling
[[4 5 7]
 [5 3 2]
 [8 2 6]] 

Mean Downsampling
[[3 4 5]
 [4 3 3]
 [6 2 4]] 



## Upsampling

In [5]:
def nearest_neighbor(matrix, scale=2):
    h,w = matrix.shape
    output_dim = (int(h*scale), int(w*scale))
    output = np.zeros(output_dim, dtype=matrix.dtype)
    stride_row = h / output_dim[0]
    stride_col = w / output_dim[1]

    for i in range(output_dim[0]):
        for j in range(output_dim[1]):
            old_i = int(i * stride_row)
            old_j = int(j * stride_col)
            output[i, j] = matrix[old_i, old_j]
    
    return output

# ...
def bilinear(matrix, scale=2):
    h, w = matrix.shape
    output_dim = (int(h * scale), int(w * scale))
    output = np.zeros(output_dim)

    for i_new in range(output_dim[0]):
        for j_new in range(output_dim[1]):
            x = i_new / scale
            y = j_new / scale
            
            x0 = int(np.floor(x))
            x1 = min(x0 + 1, h-1)
            y0 = int(np.floor(y))
            y1 = min(y0 + 1, w-1)
            
            dx = x - x0
            dy = y - y0
            
            output[i_new, j_new] = (
                matrix[x0, y0]*(1-dx)*(1-dy) +
                matrix[x1, y0]*dx*(1-dy) +
                matrix[x0, y1]*(1-dx)*dy +
                matrix[x1, y1]*dx*dy
            )

    return output

# ...
def cubic_kernel(t, a=-0.5):
    t = abs(t)
    if t <= 1:
        return (a + 2)*t**3 - (a + 3)*t**2 + 1
    elif t < 2:
        return a*t**3 - 5*a*t**2 + 8*a*t - 4*a
    else:
        return 0
    
def bicubic(matrix, scale=2):
    h, w = matrix.shape
    H_new, W_new = int(h*scale), int(w*scale)
    output = np.zeros((H_new, W_new), dtype=float)
    
    for i_new in range(H_new):
        for j_new in range(W_new):
            # Koordinat “real” di matriks asli
            x = i_new / scale
            y = j_new / scale
            x_int = int(np.floor(x))
            y_int = int(np.floor(y))
            
            value = 0
            for m in range(-1, 3):
                for n in range(-1, 3):
                    xi = min(max(x_int + m, 0), h-1)
                    yj = min(max(y_int + n, 0), w-1)
                    weight = cubic_kernel(x - (x_int + m)) * cubic_kernel(y - (y_int + n))
                    value += matrix[xi, yj] * weight
            output[i_new, j_new] = value
    return output

In [6]:
y = np.random.randint(0, 10, size=(3, 3))
print(y)

[[5 9 0]
 [6 4 0]
 [0 6 1]]


In [7]:
nn = nearest_neighbor(y)
print(f"Nearest Neighbor Upsampling")
print(nn,"\n")

bilinear = bilinear(y)
print(f"Bilinear Upsampling")
print(bilinear,"\n")

bicubic = bicubic(y)
print(f"Bicubic Upsampling")
print(bicubic,"\n")

Nearest Neighbor Upsampling
[[5 5 9 9 0 0]
 [5 5 9 9 0 0]
 [6 6 4 4 0 0]
 [6 6 4 4 0 0]
 [0 0 6 6 1 1]
 [0 0 6 6 1 1]] 

Bilinear Upsampling
[[5.   7.   9.   4.5  0.   0.  ]
 [5.5  6.   6.5  3.25 0.   0.  ]
 [6.   5.   4.   2.   0.   0.  ]
 [3.   4.   5.   2.75 0.5  0.5 ]
 [0.   3.   6.   3.5  1.   1.  ]
 [0.   3.   6.   3.5  1.   1.  ]] 

Bicubic Upsampling
[[ 5.          7.5625      9.          4.75        0.         -0.5625    ]
 [ 5.875       6.52734375  6.375       3.1875     -0.0625     -0.46484375]
 [ 6.          5.25        4.          1.875       0.         -0.25      ]
 [ 3.0625      4.13671875  4.6875      2.6953125   0.5         0.23828125]
 [ 0.          3.3125      6.          3.875       1.          0.6875    ]
 [-0.375       3.19140625  6.125       4.          1.0625      0.74609375]] 

